In [1]:
import pandas as pd
import os

In [2]:
# parcel to TAZ/SD lookup
p_taz = pd.read_csv(r'M:\Data\Urban\BAUS\PBA50\p10_parcel_TAZ_SD.csv')

print(p_taz.shape)
display(p_taz.head())

(1956208, 4)


,parcel_id,TAZ,SUPERD,SD_NAME
0,229116,725.0,15,Tri Valley
1,244166,715.0,15,Tri Valley
2,202378,820.0,17,S Leandro Hayward
3,2004420,1401.0,31,Northern Sonoma
4,340332,763.0,16,Fremont


In [3]:
# folder = 'M:\\Data\\Urban\\BAUS\\PBA50\\explore_developer_model\\runs_feasibility_all_included\\runs'
folder = r'X:\bayarea_urbansim\runs'
export = 'M:\\Data\\Urban\\BAUS\\PBA50\\explore_developer_model\\runs_new_parcel_subzone_input'
# scenarios
runs = {'run310': 'NP',
        'run313': 'P'}

## HH and residential_units data from parcel_data_[year]

In [4]:
parcel_np_df = p_taz.copy()
parcel_p_df = p_taz.copy()

for filename in os.listdir(folder):
    if (('run310_parcel_data' in filename) or ('run313_parcel_data' in filename)) & (
        'diff' not in filename) & ('parcel_data_allCols' not in filename):
        scen = runs[filename.split('_')[0]]
        yr = filename.split('.')[0][-4:]
        print(filename, scen, yr)
        df = pd.read_csv(os.path.join(folder, filename), usecols=['parcel_id', 'tothh', 'residential_units'])
        print('read {} rows of parcel output'.format(df.shape[0]))
        df.rename(columns = {'tothh': 'hh_'+yr,
                             'residential_units': 'units_'+yr}, inplace=True)
#         display(df.head())
        
        if scen == 'NP':
            parcel_np_df = parcel_np_df.merge(df, on='parcel_id', how='outer')
        elif scen == 'P':
            parcel_p_df = parcel_p_df.merge(df, on='parcel_id', how='outer')

run310_parcel_data_2010.csv NP 2010
read 1956212 rows of parcel output
run310_parcel_data_2015.csv NP 2015
read 1956212 rows of parcel output
run310_parcel_data_2020.csv NP 2020
read 1956212 rows of parcel output
run310_parcel_data_2025.csv NP 2025
read 1956212 rows of parcel output
run310_parcel_data_2030.csv NP 2030
read 1956212 rows of parcel output
run310_parcel_data_2035.csv NP 2035
read 1956212 rows of parcel output
run310_parcel_data_2040.csv NP 2040
read 1956212 rows of parcel output
run310_parcel_data_2045.csv NP 2045
read 1956212 rows of parcel output
run310_parcel_data_2050.csv NP 2050
read 1956212 rows of parcel output
run313_parcel_data_2010.csv P 2010
read 1956212 rows of parcel output
run313_parcel_data_2015.csv P 2015
read 1956212 rows of parcel output
run313_parcel_data_2020.csv P 2020
read 1956212 rows of parcel output
run313_parcel_data_2025.csv P 2025
read 1956212 rows of parcel output
run313_parcel_data_2030.csv P 2030
read 1956212 rows of parcel output
run313_parc

In [5]:
# fill NA except for parcel_id, TAZ, and SD fields

for i in list(parcel_np_df):
    if i not in ['parcel_id', 'TAZ', 'SUPERD', 'SD_NAME']:
        parcel_np_df[i].fillna(0, inplace=True)

for i in list(parcel_p_df):
    if i not in ['parcel_id', 'TAZ', 'SUPERD', 'SD_NAME']:
        parcel_p_df[i].fillna(0, inplace=True)

In [6]:
def calculate_growth_2015_base(df, metrics):
    # calculate growth relatively year 2010 value, add the new columns to df
    # df: dataframe with fields for residential_unit and tot_hh for each year
    # metrics: either 'residential_unit' or 'tot_hh'
    df_temp = df.copy()
    for i in range(2020, 2055, 5):
        df_temp[metrics+'_growth_'+str(i)] = df_temp[metrics+'_'+str(i)] - df_temp[metrics+'_2015']
    return df_temp[['parcel_id', 'TAZ', 'SUPERD', 'SD_NAME'] + [col for col in df_temp.columns if 'growth' in col]]

In [7]:
def calculate_growth_2010_base(df, metrics):
    # calculate growth relatively year 2010 value, add the new columns to df
    # df: dataframe with fields for residential_unit and tot_hh for each year
    # metrics: either 'residential_unit' or 'tot_hh'
    df_temp = df.copy()
    for i in range(2015, 2055, 5):
        df_temp[metrics+'_growth_'+str(i)] = df_temp[metrics+'_'+str(i)] - df_temp[metrics+'_2010']
    return df_temp[['parcel_id', 'TAZ', 'SUPERD', 'SD_NAME'] + [col for col in df_temp.columns if 'growth' in col]]

In [8]:
# calculate households growth and residential-units growth for all years
hh_growth_2015base_np = calculate_growth_2015_base(parcel_np_df, 'hh')
res_units_growth_2015base_np = calculate_growth_2015_base(parcel_np_df, 'units')
hh_growth_2015base_p = calculate_growth_2015_base(parcel_p_df, 'hh')
res_units_growth_2015base_p = calculate_growth_2015_base(parcel_p_df, 'units')

hh_growth_2010base_np = calculate_growth_2010_base(parcel_np_df, 'hh')
res_units_growth_2010base_np = calculate_growth_2010_base(parcel_np_df, 'units')
hh_growth_2010base_p = calculate_growth_2010_base(parcel_p_df, 'hh')
res_units_growth_2010base_p = calculate_growth_2010_base(parcel_p_df, 'units')

In [9]:
display(hh_growth_2015base_np.head())
display(hh_growth_2010base_np.head())

,parcel_id,TAZ,SUPERD,SD_NAME,hh_growth_2020,hh_growth_2025,hh_growth_2030,hh_growth_2035,hh_growth_2040,hh_growth_2045,hh_growth_2050
0,229116,725.0,15.0,Tri Valley,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,244166,715.0,15.0,Tri Valley,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,202378,820.0,17.0,S Leandro Hayward,1.0,2.0,2.0,1.0,1.0,0.0,1.0
3,2004420,1401.0,31.0,Northern Sonoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,340332,763.0,16.0,Fremont,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,parcel_id,TAZ,SUPERD,SD_NAME,hh_growth_2015,hh_growth_2020,hh_growth_2025,hh_growth_2030,hh_growth_2035,hh_growth_2040,hh_growth_2045,hh_growth_2050
0,229116,725.0,15.0,Tri Valley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,244166,715.0,15.0,Tri Valley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,202378,820.0,17.0,S Leandro Hayward,12.0,13.0,14.0,14.0,13.0,13.0,12.0,13.0
3,2004420,1401.0,31.0,Northern Sonoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,340332,763.0,16.0,Fremont,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# merge growth of NP and P
# 2015 as base year

hh_growth_2015base_np.rename(columns = dict(zip([col for col in hh_growth_2015base_np.columns if 'growth' in col],
                                                [col+'_NP' for col in hh_growth_2015base_np.columns if 'growth' in col])), inplace=True)
hh_growth_2015base_p.rename(columns = dict(zip([col for col in hh_growth_2015base_p.columns if 'growth' in col],
                                               [col+'_P' for col in hh_growth_2015base_p.columns if 'growth' in col])), inplace=True)

hh_growth_2015base_compare = hh_growth_2015base_np.merge(hh_growth_2015base_p,
                                                         on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'],
                                                         how='outer')
print(hh_growth_2015base_compare.shape[0])

res_units_growth_2015base_np.rename(columns = dict(zip([col for col in res_units_growth_2015base_np.columns if 'growth' in col],
                                                       [col+'_NP' for col in res_units_growth_2015base_np.columns if 'growth' in col])), inplace=True)
res_units_growth_2015base_p.rename(columns = dict(zip([col for col in res_units_growth_2015base_p.columns if 'growth' in col],
                                                      [col+'_P' for col in res_units_growth_2015base_p.columns if 'growth' in col])), inplace=True)

res_units_growth_2015base_compare = res_units_growth_2015base_np.merge(res_units_growth_2015base_p,
                                                                       on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'],
                                                                       how='outer')
print(res_units_growth_2015base_compare.shape[0])


1956212
1956212


In [11]:
# export parcel-level data for future reference
hh_growth_2015base_compare.to_csv(os.path.join(export, 'hh_growth_2015base_compare_byParcel.csv'), index=False)

In [12]:
# summarize by TAZ
hh_growth_2015base_compare_byTAZ = hh_growth_2015base_compare.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in hh_growth_2015base_compare.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()
res_units_growth_2015base_compare_byTAZ = res_units_growth_2015base_compare.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in res_units_growth_2015base_compare.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()

# export
hh_growth_2015base_compare_byTAZ.to_csv(os.path.join(export, 'hh_growth_2015base_compare_byTAZ.csv'), index=False)
res_units_growth_2015base_compare_byTAZ.to_csv(os.path.join(export, 'res_units_growth_2015base_compare_byTAZ.csv'), index=False)

In [13]:
# merge growth of NP and P
# 2010 as base year

hh_growth_2010base_np.rename(columns = dict(zip([col for col in hh_growth_2010base_np.columns if 'growth' in col],
                                                [col+'_NP' for col in hh_growth_2010base_np.columns if 'growth' in col])), inplace=True)
hh_growth_2010base_p.rename(columns = dict(zip([col for col in hh_growth_2010base_p.columns if 'growth' in col],
                                               [col+'_P' for col in hh_growth_2010base_p.columns if 'growth' in col])), inplace=True)

hh_growth_2010base_compare = hh_growth_2010base_np.merge(hh_growth_2010base_p,
                                                         on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'],
                                                         how='outer')
print(hh_growth_2010base_compare.shape[0])

res_units_growth_2010base_np.rename(columns = dict(zip([col for col in res_units_growth_2010base_np.columns if 'growth' in col],
                                                       [col+'_NP' for col in res_units_growth_2010base_np.columns if 'growth' in col])), inplace=True)
res_units_growth_2010base_p.rename(columns = dict(zip([col for col in res_units_growth_2010base_p.columns if 'growth' in col],
                                                      [col+'_P' for col in res_units_growth_2010base_p.columns if 'growth' in col])), inplace=True)

res_units_growth_2010base_compare = res_units_growth_2010base_np.merge(res_units_growth_2010base_p,
                                                                       on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'],
                                                                       how='outer')
print(res_units_growth_2010base_compare.shape[0])

1956212
1956212


In [14]:
# summarize by TAZ
hh_growth_2010base_compare_byTAZ = hh_growth_2010base_compare.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in hh_growth_2010base_compare.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()
res_units_growth_2010base_compare_byTAZ = res_units_growth_2010base_compare.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in res_units_growth_2010base_compare.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()

# export
hh_growth_2010base_compare_byTAZ.to_csv(os.path.join(export, 'hh_growth_2010base_compare_byTAZ.csv'), index=False)
res_units_growth_2010base_compare_byTAZ.to_csv(os.path.join(export, 'res_units_growth_2010base_compare_byTAZ.csv'), index=False)

## building_data_[year] with information on source of growth

In [15]:
def count_units_by_source_from_blg(blg_df, year):
# df = pd.read_csv(r'M:\Data\Urban\BAUS\PBA50\explore_developer_model\runs\run264_building_data_2050.csv',
#                  usecols=['building_id', 'parcel_id', 'residential_units', 'source'])

    df_dev = blg_df.loc[blg_df.source == 'developer_model']
    df_dev = df_dev[['parcel_id', 'residential_units']].rename(
        columns = {'residential_units': 'units_dev_'+year})
    df_parcel_dev = df_dev.groupby('parcel_id').sum().reset_index()

    df_nondev = blg_df.loc[blg_df.source != 'developer_model']
    df_nondev = df_nondev[['parcel_id','residential_units']].rename(
        columns = {'residential_units': 'units_nondev_'+year})
    df_parcel_nondev = df_nondev.groupby('parcel_id').sum().reset_index()

    df_parcel = df_parcel_dev.merge(df_parcel_nondev, on='parcel_id', how='outer')
    
    return df_parcel


In [16]:
parcel_blg_np_df = p_taz.copy()
parcel_blg_p_df = p_taz.copy()

for filename in os.listdir(folder):
    if ('run310_building_data' in filename) or ('run313_building_data' in filename):
        scen = runs[filename.split('_')[0]]
        yr = filename.split('.')[0][-4:]
        print(filename, scen, yr)

        df = pd.read_csv(os.path.join(folder, filename), usecols=['building_id', 'parcel_id',
                                                                  'residential_units', 'source'])
        print('read {} rows of building output on {} parcels'.format(df.shape[0], len(df.parcel_id.unique())))
        
        units_by_source = count_units_by_source_from_blg(df, yr)
        print('{} rows of parcel-level data after summarizing by source'.format(units_by_source.shape[0]))
        
        if scen == 'NP':
            parcel_blg_np_df = parcel_blg_np_df.merge(units_by_source, on='parcel_id', how='outer')
        elif scen == 'P':
            parcel_blg_p_df = parcel_blg_p_df.merge(units_by_source, on='parcel_id', how='outer')


run310_building_data_2010.csv NP 2010
read 1843351 rows of building output on 1843292 parcels
1843292 rows of parcel-level data after summarizing by source
run310_building_data_2015.csv NP 2015
read 1859454 rows of building output on 1846939 parcels
1846939 rows of parcel-level data after summarizing by source
run310_building_data_2020.csv NP 2020
read 1869727 rows of building output on 1848247 parcels
1848247 rows of parcel-level data after summarizing by source
run310_building_data_2025.csv NP 2025
read 1870233 rows of building output on 1848387 parcels
1848387 rows of parcel-level data after summarizing by source
run310_building_data_2030.csv NP 2030
read 1871043 rows of building output on 1848904 parcels
1848904 rows of parcel-level data after summarizing by source
run310_building_data_2035.csv NP 2035
read 1871076 rows of building output on 1848644 parcels
1848644 rows of parcel-level data after summarizing by source
run310_building_data_2040.csv NP 2040
read 1871895 rows of build

In [17]:
# fill NA except for parcel_id, TAZ, and SD fields

for i in list(parcel_blg_np_df):
    if i not in ['parcel_id', 'TAZ', 'SUPERD', 'SD_NAME']:
        parcel_blg_np_df[i].fillna(0, inplace=True)

for i in list(parcel_blg_p_df):
    if i not in ['parcel_id', 'TAZ', 'SUPERD', 'SD_NAME']:
        parcel_blg_p_df[i].fillna(0, inplace=True)

In [18]:
# calculate growth of units from developer model and grwoth of units not from developer model
units_growth_dev_2015base_np = calculate_growth_2015_base(parcel_blg_np_df, 'units_dev')
units_growth_nondev_2015base_np = calculate_growth_2015_base(parcel_blg_np_df, 'units_nondev')
units_growth_dev_2015base_p = calculate_growth_2015_base(parcel_blg_p_df, 'units_dev')
units_growth_nondev_2015base_p = calculate_growth_2015_base(parcel_blg_p_df, 'units_nondev')

units_growth_dev_2010base_np = calculate_growth_2010_base(parcel_blg_np_df, 'units_dev')
units_growth_nondev_2010base_np = calculate_growth_2010_base(parcel_blg_np_df, 'units_nondev')
units_growth_dev_2010base_p = calculate_growth_2010_base(parcel_blg_p_df, 'units_dev')
units_growth_nondev_2010base_p = calculate_growth_2010_base(parcel_blg_p_df, 'units_nondev')

In [19]:
display(units_growth_dev_2015base_np.head())
display(units_growth_nondev_2015base_np.head())

,parcel_id,TAZ,SUPERD,SD_NAME,units_dev_growth_2020,units_dev_growth_2025,units_dev_growth_2030,units_dev_growth_2035,units_dev_growth_2040,units_dev_growth_2045,units_dev_growth_2050
0,229116,725.0,15.0,Tri Valley,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,244166,715.0,15.0,Tri Valley,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,202378,820.0,17.0,S Leandro Hayward,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2004420,1401.0,31.0,Northern Sonoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,340332,763.0,16.0,Fremont,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,parcel_id,TAZ,SUPERD,SD_NAME,units_nondev_growth_2020,units_nondev_growth_2025,units_nondev_growth_2030,units_nondev_growth_2035,units_nondev_growth_2040,units_nondev_growth_2045,units_nondev_growth_2050
0,229116,725.0,15.0,Tri Valley,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,244166,715.0,15.0,Tri Valley,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,202378,820.0,17.0,S Leandro Hayward,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2004420,1401.0,31.0,Northern Sonoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,340332,763.0,16.0,Fremont,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# merge growth of NP and P
# 2015 as base year

units_growth_dev_2015base_np_scen = units_growth_dev_2015base_np.rename(columns = dict(zip([col for col in units_growth_dev_2015base_np.columns if 'growth' in col],
                                                [col+'_NP' for col in units_growth_dev_2015base_np.columns if 'growth' in col])))
print(units_growth_dev_2015base_np_scen.shape[0])
units_growth_dev_2015base_p_scen = units_growth_dev_2015base_p.rename(columns = dict(zip([col for col in units_growth_dev_2015base_p.columns if 'growth' in col],
                                               [col+'_P' for col in units_growth_dev_2015base_p.columns if 'growth' in col])))
print(units_growth_dev_2015base_p_scen.shape[0])


units_growth_nondev_2015base_np_scen = units_growth_nondev_2015base_np.rename(columns = dict(zip([col for col in units_growth_nondev_2015base_np.columns if 'growth' in col],
                                                       [col+'_NP' for col in units_growth_nondev_2015base_np.columns if 'growth' in col])))
print(units_growth_nondev_2015base_np_scen.shape[0])
units_growth_nondev_2015base_p_scen = units_growth_nondev_2015base_p.rename(columns = dict(zip([col for col in units_growth_nondev_2015base_p.columns if 'growth' in col],
                                                      [col+'_P' for col in units_growth_nondev_2015base_p.columns if 'growth' in col])))
print(units_growth_nondev_2015base_p_scen.shape[0])

# merge
units_growth_bySource_2015base_compare = units_growth_dev_2015base_np_scen.merge(
                                         units_growth_dev_2015base_p_scen, on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'], how='outer').merge(
                                         units_growth_nondev_2015base_np_scen, on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'], how='outer').merge(
                                         units_growth_nondev_2015base_p_scen, on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'], how='outer')

print(units_growth_bySource_2015base_compare.shape[0])
print(len(units_growth_bySource_2015base_compare.parcel_id.unique()))
print(list(units_growth_bySource_2015base_compare))

1956210
1956210
1956210
1956210
1956210
1956210
['parcel_id', 'TAZ', 'SUPERD', 'SD_NAME', 'units_dev_growth_2020_NP', 'units_dev_growth_2025_NP', 'units_dev_growth_2030_NP', 'units_dev_growth_2035_NP', 'units_dev_growth_2040_NP', 'units_dev_growth_2045_NP', 'units_dev_growth_2050_NP', 'units_dev_growth_2020_P', 'units_dev_growth_2025_P', 'units_dev_growth_2030_P', 'units_dev_growth_2035_P', 'units_dev_growth_2040_P', 'units_dev_growth_2045_P', 'units_dev_growth_2050_P', 'units_nondev_growth_2020_NP', 'units_nondev_growth_2025_NP', 'units_nondev_growth_2030_NP', 'units_nondev_growth_2035_NP', 'units_nondev_growth_2040_NP', 'units_nondev_growth_2045_NP', 'units_nondev_growth_2050_NP', 'units_nondev_growth_2020_P', 'units_nondev_growth_2025_P', 'units_nondev_growth_2030_P', 'units_nondev_growth_2035_P', 'units_nondev_growth_2040_P', 'units_nondev_growth_2045_P', 'units_nondev_growth_2050_P']


In [21]:
# summarize by TAZ - stack easier for Tableau

# NP
units_growth_dev_2015base_np_byTAZ = units_growth_dev_2015base_np.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in units_growth_dev_2015base_np.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()
units_growth_nondev_2015base_np_byTAZ = units_growth_nondev_2015base_np.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in units_growth_nondev_2015base_np.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()


units_growth_dev_2015base_np_byTAZ.rename(columns = dict(zip([col for col in units_growth_dev_2015base_np_byTAZ.columns if 'growth' in col],
                                                [col.replace('_dev_', '_') for col in units_growth_dev_2015base_np_byTAZ.columns if 'growth' in col])), inplace=True)
units_growth_dev_2015base_np_byTAZ['source'] = 'developer model'
units_growth_dev_2015base_np_byTAZ['scen'] = 'NP'

units_growth_nondev_2015base_np_byTAZ.rename(columns = dict(zip([col for col in units_growth_nondev_2015base_np_byTAZ.columns if 'growth' in col],
                                                [col.replace('_nondev_', '_') for col in units_growth_nondev_2015base_np_byTAZ.columns if 'growth' in col])), inplace=True)
units_growth_nondev_2015base_np_byTAZ['source'] = 'non-developer model'
units_growth_nondev_2015base_np_byTAZ['scen'] = 'NP'


# P
units_growth_dev_2015base_p_byTAZ = units_growth_dev_2015base_p.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in units_growth_dev_2015base_p.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()
units_growth_nondev_2015base_p_byTAZ = units_growth_nondev_2015base_p.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in units_growth_nondev_2015base_p.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()


units_growth_dev_2015base_p_byTAZ.rename(columns = dict(zip([col for col in units_growth_dev_2015base_p_byTAZ.columns if 'growth' in col],
                                                [col.replace('_dev_', '_') for col in units_growth_dev_2015base_p_byTAZ.columns if 'growth' in col])), inplace=True)
units_growth_dev_2015base_p_byTAZ['source'] = 'developer model'
units_growth_dev_2015base_p_byTAZ['scen'] = 'P'

units_growth_nondev_2015base_p_byTAZ.rename(columns = dict(zip([col for col in units_growth_nondev_2015base_p_byTAZ.columns if 'growth' in col],
                                                [col.replace('_nondev_', '_') for col in units_growth_nondev_2015base_p_byTAZ.columns if 'growth' in col])), inplace=True)
units_growth_nondev_2015base_p_byTAZ['source'] = 'non-developer model'
units_growth_nondev_2015base_p_byTAZ['scen'] = 'P'


# merge
units_growth_bySource_byTAZ_2015base_compare = pd.concat([units_growth_dev_2015base_np_byTAZ,
                                                          units_growth_nondev_2015base_np_byTAZ,
                                                          units_growth_dev_2015base_p_byTAZ,
                                                          units_growth_nondev_2015base_p_byTAZ])

units_growth_bySource_byTAZ_2015base_compare.to_csv(os.path.join(export, 'units_growth_bySource_byTAZ_2015base_compare.csv'), index=False)

In [22]:
# merge growth of NP and P
# 2010 as base year

units_growth_dev_2010base_np_scen = units_growth_dev_2010base_np.rename(columns = dict(zip([col for col in units_growth_dev_2010base_np.columns if 'growth' in col],
                                                [col+'_NP' for col in units_growth_dev_2010base_np.columns if 'growth' in col])))
print(units_growth_dev_2010base_np_scen.shape[0])
units_growth_dev_2010base_p_scen = units_growth_dev_2010base_p.rename(columns = dict(zip([col for col in units_growth_dev_2010base_p.columns if 'growth' in col],
                                               [col+'_P' for col in units_growth_dev_2010base_p.columns if 'growth' in col])))
print(units_growth_dev_2010base_p_scen.shape[0])


units_growth_nondev_2010base_np_scen = units_growth_nondev_2010base_np.rename(columns = dict(zip([col for col in units_growth_nondev_2010base_np.columns if 'growth' in col],
                                                       [col+'_NP' for col in units_growth_nondev_2010base_np.columns if 'growth' in col])))
print(units_growth_nondev_2010base_np_scen.shape[0])
units_growth_nondev_2010base_p_scen = units_growth_nondev_2010base_p.rename(columns = dict(zip([col for col in units_growth_nondev_2010base_p.columns if 'growth' in col],
                                                      [col+'_P' for col in units_growth_nondev_2010base_p.columns if 'growth' in col])))
print(units_growth_nondev_2010base_p_scen.shape[0])

# merge
units_growth_bySource_2010base_compare = units_growth_dev_2010base_np_scen.merge(
                                         units_growth_dev_2010base_p_scen, on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'], how='outer').merge(
                                         units_growth_nondev_2010base_np_scen, on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'], how='outer').merge(
                                         units_growth_nondev_2010base_p_scen, on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'], how='outer')

print(units_growth_bySource_2010base_compare.shape[0])
print(len(units_growth_bySource_2010base_compare.parcel_id.unique()))
print(list(units_growth_bySource_2010base_compare))

1956210
1956210
1956210
1956210
1956210
1956210
['parcel_id', 'TAZ', 'SUPERD', 'SD_NAME', 'units_dev_growth_2015_NP', 'units_dev_growth_2020_NP', 'units_dev_growth_2025_NP', 'units_dev_growth_2030_NP', 'units_dev_growth_2035_NP', 'units_dev_growth_2040_NP', 'units_dev_growth_2045_NP', 'units_dev_growth_2050_NP', 'units_dev_growth_2015_P', 'units_dev_growth_2020_P', 'units_dev_growth_2025_P', 'units_dev_growth_2030_P', 'units_dev_growth_2035_P', 'units_dev_growth_2040_P', 'units_dev_growth_2045_P', 'units_dev_growth_2050_P', 'units_nondev_growth_2015_NP', 'units_nondev_growth_2020_NP', 'units_nondev_growth_2025_NP', 'units_nondev_growth_2030_NP', 'units_nondev_growth_2035_NP', 'units_nondev_growth_2040_NP', 'units_nondev_growth_2045_NP', 'units_nondev_growth_2050_NP', 'units_nondev_growth_2015_P', 'units_nondev_growth_2020_P', 'units_nondev_growth_2025_P', 'units_nondev_growth_2030_P', 'units_nondev_growth_2035_P', 'units_nondev_growth_2040_P', 'units_nondev_growth_2045_P', 'units_nonde

In [23]:
# summarize by TAZ - stack easier for Tableau

# NP
units_growth_dev_2010base_np_byTAZ = units_growth_dev_2010base_np.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in units_growth_dev_2010base_np.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()
units_growth_nondev_2010base_np_byTAZ = units_growth_nondev_2010base_np.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in units_growth_nondev_2010base_np.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()


units_growth_dev_2010base_np_byTAZ.rename(columns = dict(zip([col for col in units_growth_dev_2010base_np_byTAZ.columns if 'growth' in col],
                                                [col.replace('_dev_', '_') for col in units_growth_dev_2010base_np_byTAZ.columns if 'growth' in col])), inplace=True)
units_growth_dev_2010base_np_byTAZ['source'] = 'developer model'
units_growth_dev_2010base_np_byTAZ['scen'] = 'NP'

units_growth_nondev_2010base_np_byTAZ.rename(columns = dict(zip([col for col in units_growth_nondev_2010base_np_byTAZ.columns if 'growth' in col],
                                                [col.replace('_nondev_', '_') for col in units_growth_nondev_2010base_np_byTAZ.columns if 'growth' in col])), inplace=True)
units_growth_nondev_2010base_np_byTAZ['source'] = 'non-developer model'
units_growth_nondev_2010base_np_byTAZ['scen'] = 'NP'


# P
units_growth_dev_2010base_p_byTAZ = units_growth_dev_2010base_p.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in units_growth_dev_2010base_p.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()
units_growth_nondev_2010base_p_byTAZ = units_growth_nondev_2010base_p.groupby(
    ['SD_NAME', 'SUPERD', 'TAZ'])[[col for col in units_growth_nondev_2010base_p.columns if 'growth' in col]].sum().sort_index(axis=1).reset_index()


units_growth_dev_2010base_p_byTAZ.rename(columns = dict(zip([col for col in units_growth_dev_2010base_p_byTAZ.columns if 'growth' in col],
                                                [col.replace('_dev_', '_') for col in units_growth_dev_2010base_p_byTAZ.columns if 'growth' in col])), inplace=True)
units_growth_dev_2010base_p_byTAZ['source'] = 'developer model'
units_growth_dev_2010base_p_byTAZ['scen'] = 'P'

units_growth_nondev_2010base_p_byTAZ.rename(columns = dict(zip([col for col in units_growth_nondev_2010base_p_byTAZ.columns if 'growth' in col],
                                                [col.replace('_nondev_', '_') for col in units_growth_nondev_2010base_p_byTAZ.columns if 'growth' in col])), inplace=True)
units_growth_nondev_2010base_p_byTAZ['source'] = 'non-developer model'
units_growth_nondev_2010base_p_byTAZ['scen'] = 'P'


# merge
units_growth_bySource_byTAZ_2010base_compare = pd.concat([units_growth_dev_2010base_np_byTAZ,
                                                          units_growth_nondev_2010base_np_byTAZ,
                                                          units_growth_dev_2010base_p_byTAZ,
                                                          units_growth_nondev_2010base_p_byTAZ])

units_growth_bySource_byTAZ_2010base_compare.to_csv(os.path.join(export, 'units_growth_bySource_byTAZ_2010base_compare.csv'), index=False)

## SE Snta Clara Cnty, 2015 as base year

In [24]:
hh_compare_sescl_2015base = hh_growth_2015base_compare.loc[
    hh_growth_2015base_compare.SD_NAME == 'SE Snta Clara Cnty']

unit_compare_sescl_2015base = res_units_growth_2015base_compare.loc[
    res_units_growth_2015base_compare.SD_NAME == 'SE Snta Clara Cnty']

compare_sescl_2015base = hh_compare_sescl_2015base.merge(unit_compare_sescl_2015base,
                                                         on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'],
                                                         how='outer')

for year in range(2020, 2055, 5):
    print('year {}\n'.format(year))

    hh_P_more = compare_sescl_2015base.loc[compare_sescl_2015base['hh_growth_'+str(year)+'_P'] > compare_sescl_2015base['hh_growth_'+str(year)+'_NP']]
    hh_NP_more = compare_sescl_2015base.loc[compare_sescl_2015base['hh_growth_'+str(year)+'_P'] < compare_sescl_2015base['hh_growth_'+str(year)+'_NP']]
    hh_equal = compare_sescl_2015base.loc[compare_sescl_2015base['hh_growth_'+str(year)+'_P'] == compare_sescl_2015base['hh_growth_'+str(year)+'_NP']]
    print('{} parcels have households growth P > NP: \n{}\n'.format(hh_P_more.shape[0],
                                                                    hh_P_more[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                               'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have households growth P < NP: \n{}\n'.format(hh_NP_more.shape[0],
                                                                    hh_NP_more[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                                'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have households growth P = NP: \n{}\n'.format(hh_equal.shape[0],
                                                                    hh_equal[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                              'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))
    
    unit_P_more = compare_sescl_2015base.loc[compare_sescl_2015base['units_growth_'+str(year)+'_P'] > compare_sescl_2015base['units_growth_'+str(year)+'_NP']]
    unit_NP_more = compare_sescl_2015base.loc[compare_sescl_2015base['units_growth_'+str(year)+'_P'] < compare_sescl_2015base['units_growth_'+str(year)+'_NP']]
    unit_equal = compare_sescl_2015base.loc[compare_sescl_2015base['units_growth_'+str(year)+'_P'] == compare_sescl_2015base['units_growth_'+str(year)+'_NP']]
    print('{} parcels have residential unit growth P > NP: \n{}\n'.format(unit_P_more.shape[0],
                                                                          unit_P_more[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                              'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P < NP: \n{}\n'.format(unit_NP_more.shape[0],
                                                                          unit_NP_more[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                                        'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P = NP: \n{}\n\n'.format(unit_equal.shape[0],
                                                                            unit_equal[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                                        'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))

year 2020

2789 parcels have households growth P > NP: 
hh_growth_2020_NP      -1146.0
hh_growth_2020_P        2041.0
units_growth_2020_NP     600.0
units_growth_2020_P      630.0
dtype: float64

2847 parcels have households growth P < NP: 
hh_growth_2020_NP       2320.0
hh_growth_2020_P        -922.0
units_growth_2020_NP     724.0
units_growth_2020_P      700.0
dtype: float64

28401 parcels have households growth P = NP: 
hh_growth_2020_NP       332.0
hh_growth_2020_P        332.0
units_growth_2020_NP    504.0
units_growth_2020_P     501.0
dtype: float64

49 parcels have residential unit growth P > NP: 
hh_growth_2020_NP       146.0
hh_growth_2020_P        166.0
units_growth_2020_NP     57.0
units_growth_2020_P     106.0
dtype: float64

46 parcels have residential unit growth P < NP: 
hh_growth_2020_NP       79.0
hh_growth_2020_P        44.0
units_growth_2020_NP    93.0
units_growth_2020_P     47.0
dtype: float64

33942 parcels have residential unit growth P = NP: 
hh_growth_2020_NP  

In [25]:
src_compare_sescl_2015base = units_growth_bySource_2015base_compare.loc[
    units_growth_bySource_2015base_compare.SD_NAME == 'SE Snta Clara Cnty']

for year in range(2020, 2055, 5):
    print('year ', year)

    unit_dev_P_more = src_compare_sescl_2015base.loc[src_compare_sescl_2015base['units_dev_growth_'+str(year)+'_P'] > src_compare_sescl_2015base['units_dev_growth_'+str(year)+'_NP']]
    unit_dev_NP_more = src_compare_sescl_2015base.loc[src_compare_sescl_2015base['units_dev_growth_'+str(year)+'_P'] < src_compare_sescl_2015base['units_dev_growth_'+str(year)+'_NP']]
    unit_dev_equal = src_compare_sescl_2015base.loc[src_compare_sescl_2015base['units_dev_growth_'+str(year)+'_P'] == src_compare_sescl_2015base['units_dev_growth_'+str(year)+'_NP']]
    print('{} parcels have residential unit growth P > NP from Developer Model: \n{}\n'.format(unit_dev_P_more.shape[0],
                                                                                               unit_dev_P_more[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                                'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P < NP from Developer Model: \n{}\n'.format(unit_dev_NP_more.shape[0],
                                                                                               unit_dev_NP_more[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                                 'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P = NP from Developer Model: \n{}\n'.format(unit_dev_equal.shape[0],
                                                                                               unit_dev_equal[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                               'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))
    
    unit_nondev_P_more = src_compare_sescl_2015base.loc[src_compare_sescl_2015base['units_nondev_growth_'+str(year)+'_P'] > src_compare_sescl_2015base['units_nondev_growth_'+str(year)+'_NP']]
    unit_nondev_NP_more = src_compare_sescl_2015base.loc[src_compare_sescl_2015base['units_nondev_growth_'+str(year)+'_P'] < src_compare_sescl_2015base['units_nondev_growth_'+str(year)+'_NP']]
    unit_nondev_equal = src_compare_sescl_2015base.loc[src_compare_sescl_2015base['units_nondev_growth_'+str(year)+'_P'] == src_compare_sescl_2015base['units_nondev_growth_'+str(year)+'_NP']]
    print('{} parcels have residential unit growth P > NP from Pipeline: \n{}\n'.format(unit_nondev_P_more.shape[0],
                                                                                        unit_nondev_P_more[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                             'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P < NP from Pipeline: \n{}\n'.format(unit_nondev_NP_more.shape[0],
                                                                                        unit_nondev_NP_more[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                             'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P = NP from Pipeline: \n{}\n\n'.format(unit_nondev_equal.shape[0],
                                                                                          unit_nondev_equal[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                             'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))

year  2020
0 parcels have residential unit growth P > NP from Developer Model: 
units_dev_growth_2020_NP       0.0
units_dev_growth_2020_P        0.0
units_nondev_growth_2020_NP    0.0
units_nondev_growth_2020_P     0.0
dtype: float64

0 parcels have residential unit growth P < NP from Developer Model: 
units_dev_growth_2020_NP       0.0
units_dev_growth_2020_P        0.0
units_nondev_growth_2020_NP    0.0
units_nondev_growth_2020_P     0.0
dtype: float64

34037 parcels have residential unit growth P = NP from Developer Model: 
units_dev_growth_2020_NP          0.0
units_dev_growth_2020_P           0.0
units_nondev_growth_2020_NP    1828.0
units_nondev_growth_2020_P     1831.0
dtype: float64

49 parcels have residential unit growth P > NP from Pipeline: 
units_dev_growth_2020_NP         0.0
units_dev_growth_2020_P          0.0
units_nondev_growth_2020_NP     57.0
units_nondev_growth_2020_P     106.0
dtype: float64

46 parcels have residential unit growth P < NP from Pipeline: 
units_de

## SE Snta Clara Cnty, 2010 as base year

In [26]:
hh_compare_sescl_2010base = hh_growth_2010base_compare.loc[
    hh_growth_2010base_compare.SD_NAME == 'SE Snta Clara Cnty']

unit_compare_sescl_2010base = res_units_growth_2010base_compare.loc[
    res_units_growth_2010base_compare.SD_NAME == 'SE Snta Clara Cnty']

compare_sescl_2010base = hh_compare_sescl_2010base.merge(unit_compare_sescl_2010base,
                                                         on=['SD_NAME', 'SUPERD', 'TAZ', 'parcel_id'],
                                                         how='outer')

for year in range(2015, 2055, 5):
    print('year {}\n'.format(year))

    hh_P_more = compare_sescl_2010base.loc[compare_sescl_2010base['hh_growth_'+str(year)+'_P'] > compare_sescl_2010base['hh_growth_'+str(year)+'_NP']]
    hh_NP_more = compare_sescl_2010base.loc[compare_sescl_2010base['hh_growth_'+str(year)+'_P'] < compare_sescl_2010base['hh_growth_'+str(year)+'_NP']]
    hh_equal = compare_sescl_2010base.loc[compare_sescl_2010base['hh_growth_'+str(year)+'_P'] == compare_sescl_2010base['hh_growth_'+str(year)+'_NP']]
    print('{} parcels have households growth P > NP: \n{}\n'.format(hh_P_more.shape[0],
                                                                    hh_P_more[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                               'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have households growth P < NP: \n{}\n'.format(hh_NP_more.shape[0],
                                                                    hh_NP_more[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                                'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have households growth P = NP: \n{}\n'.format(hh_equal.shape[0],
                                                                    hh_equal[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                              'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))
    
    unit_P_more = compare_sescl_2010base.loc[compare_sescl_2010base['units_growth_'+str(year)+'_P'] > compare_sescl_2010base['units_growth_'+str(year)+'_NP']]
    unit_NP_more = compare_sescl_2010base.loc[compare_sescl_2010base['units_growth_'+str(year)+'_P'] < compare_sescl_2010base['units_growth_'+str(year)+'_NP']]
    unit_equal = compare_sescl_2010base.loc[compare_sescl_2010base['units_growth_'+str(year)+'_P'] == compare_sescl_2010base['units_growth_'+str(year)+'_NP']]
    print('{} parcels have residential unit growth P > NP: \n{}\n'.format(unit_P_more.shape[0],
                                                                          unit_P_more[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                              'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P < NP: \n{}\n'.format(unit_NP_more.shape[0],
                                                                          unit_NP_more[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                                        'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P = NP: \n{}\n\n'.format(unit_equal.shape[0],
                                                                            unit_equal[['hh_growth_'+str(year)+'_NP', 'hh_growth_'+str(year)+'_P',
                                                                                        'units_growth_'+str(year)+'_NP', 'units_growth_'+str(year)+'_P']].sum()))

year 2015

2461 parcels have households growth P > NP: 
hh_growth_2015_NP       -264.0
hh_growth_2015_P        2386.0
units_growth_2015_NP    1004.0
units_growth_2015_P     1037.0
dtype: float64

2531 parcels have households growth P < NP: 
hh_growth_2015_NP       2258.0
hh_growth_2015_P        -444.0
units_growth_2015_NP     791.0
units_growth_2015_P      755.0
dtype: float64

29045 parcels have households growth P = NP: 
hh_growth_2015_NP       528.0
hh_growth_2015_P        528.0
units_growth_2015_NP    644.0
units_growth_2015_P     643.0
dtype: float64

45 parcels have residential unit growth P > NP: 
hh_growth_2015_NP       348.0
hh_growth_2015_P        395.0
units_growth_2015_NP    521.0
units_growth_2015_P     568.0
dtype: float64

51 parcels have residential unit growth P < NP: 
hh_growth_2015_NP       64.0
hh_growth_2015_P        24.0
units_growth_2015_NP    75.0
units_growth_2015_P     24.0
dtype: float64

33941 parcels have residential unit growth P = NP: 
hh_growth_2015_NP  

In [27]:
src_compare_sescl_2010base = units_growth_bySource_2010base_compare.loc[
    units_growth_bySource_2010base_compare.SD_NAME == 'SE Snta Clara Cnty']

for year in range(2015, 2055, 5):
    print('year ', year)

    unit_dev_P_more = src_compare_sescl_2010base.loc[src_compare_sescl_2010base['units_dev_growth_'+str(year)+'_P'] > src_compare_sescl_2010base['units_dev_growth_'+str(year)+'_NP']]
    unit_dev_NP_more = src_compare_sescl_2010base.loc[src_compare_sescl_2010base['units_dev_growth_'+str(year)+'_P'] < src_compare_sescl_2010base['units_dev_growth_'+str(year)+'_NP']]
    unit_dev_equal = src_compare_sescl_2010base.loc[src_compare_sescl_2010base['units_dev_growth_'+str(year)+'_P'] == src_compare_sescl_2010base['units_dev_growth_'+str(year)+'_NP']]
    print('{} parcels have residential unit growth P > NP from Developer Model: \n{}\n'.format(unit_dev_P_more.shape[0],
                                                                                               unit_dev_P_more[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                                'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P < NP from Developer Model: \n{}\n'.format(unit_dev_NP_more.shape[0],
                                                                                               unit_dev_NP_more[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                                 'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P = NP from Developer Model: \n{}\n'.format(unit_dev_equal.shape[0],
                                                                                               unit_dev_equal[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                               'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))
    
    unit_nondev_P_more = src_compare_sescl_2010base.loc[src_compare_sescl_2010base['units_nondev_growth_'+str(year)+'_P'] > src_compare_sescl_2010base['units_nondev_growth_'+str(year)+'_NP']]
    unit_nondev_NP_more = src_compare_sescl_2010base.loc[src_compare_sescl_2010base['units_nondev_growth_'+str(year)+'_P'] < src_compare_sescl_2010base['units_nondev_growth_'+str(year)+'_NP']]
    unit_nondev_equal = src_compare_sescl_2010base.loc[src_compare_sescl_2010base['units_nondev_growth_'+str(year)+'_P'] == src_compare_sescl_2010base['units_nondev_growth_'+str(year)+'_NP']]
    print('{} parcels have residential unit growth P > NP from Pipeline: \n{}\n'.format(unit_nondev_P_more.shape[0],
                                                                                        unit_nondev_P_more[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                             'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P < NP from Pipeline: \n{}\n'.format(unit_nondev_NP_more.shape[0],
                                                                                        unit_nondev_NP_more[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                             'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))
    print('{} parcels have residential unit growth P = NP from Pipeline: \n{}\n\n'.format(unit_nondev_equal.shape[0],
                                                                                          unit_nondev_equal[['units_dev_growth_'+str(year)+'_NP', 'units_dev_growth_'+str(year)+'_P',
                                                                                                             'units_nondev_growth_'+str(year)+'_NP', 'units_nondev_growth_'+str(year)+'_P']].sum()))

year  2015
0 parcels have residential unit growth P > NP from Developer Model: 
units_dev_growth_2015_NP       0.0
units_dev_growth_2015_P        0.0
units_nondev_growth_2015_NP    0.0
units_nondev_growth_2015_P     0.0
dtype: float64

0 parcels have residential unit growth P < NP from Developer Model: 
units_dev_growth_2015_NP       0.0
units_dev_growth_2015_P        0.0
units_nondev_growth_2015_NP    0.0
units_nondev_growth_2015_P     0.0
dtype: float64

34037 parcels have residential unit growth P = NP from Developer Model: 
units_dev_growth_2015_NP          0.0
units_dev_growth_2015_P           0.0
units_nondev_growth_2015_NP    2439.0
units_nondev_growth_2015_P     2435.0
dtype: float64

45 parcels have residential unit growth P > NP from Pipeline: 
units_dev_growth_2015_NP         0.0
units_dev_growth_2015_P          0.0
units_nondev_growth_2015_NP    521.0
units_nondev_growth_2015_P     568.0
dtype: float64

51 parcels have residential unit growth P < NP from Pipeline: 
units_de